In [1]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras
import os.path
from saveDataset import save_data

# Helper libraries
import numpy as np
from numpy import genfromtxt
import matplotlib.pyplot as plt
import glob, os
import re

# Pillow
import PIL
from PIL import Image

set rootpath= path of the directory containing 'image-recognition-tensorflow-master'

In [2]:
rootpath='D:\hackathon'

In [3]:
# Use Pillow library to convert an input jpeg to a 8 bit grey scale image array for processing.
def jpeg_to_8_bit_greyscale(path, maxsize):
        img = Image.open(path).convert('L')   # convert image to 8-bit grayscale
        # Make aspect ratio as 1:1, by applying image crop.
    # Please note, croping works for this data set, but in general one
    # needs to locate the subject and then crop or scale accordingly.
        WIDTH, HEIGHT = img.size
        if WIDTH != HEIGHT:
                m_min_d = min(WIDTH, HEIGHT)
                img = img.crop((0, 0, m_min_d, m_min_d))
        # Scale the image to the requested maxsize by Anti-alias sampling.
        img.thumbnail(maxsize, PIL.Image.ANTIALIAS)
        return np.asarray(img)

In [4]:
def load_image_dataset(path_dir, maxsize):
        images = []
        labels = []
        os.chdir(path_dir)
        for file in glob.glob("*.jpg"):
                img = jpeg_to_8_bit_greyscale(file, maxsize)
#                 if re.match('chihuahua.*', file):
                if re.match('plastic.*', file):
                        images.append(img)
                        labels.append(0)
#                 elif re.match('muffin.*', file):
                elif re.match('paper.*', file):
                        images.append(img)
                        labels.append(1)
                else:
                    images.append(img)
                    labels.append(1)
        return (np.array(images), np.array(labels))

In [5]:
def shuffle_examples(X,Y):
    dim= X.shape
    X= np.reshape(X,(X.shape[0],-1))
    Y=Y.reshape(-1,1)
    X= np.append(X, Y, axis=1)
    
    np.random.seed(0)
    np.random.shuffle(X)
    
    A=np.array([])
    
    for i in range(X.shape[1]-1):
        if i==0:
            A= np.array(X[:,i]).reshape(-1,1)
        else:
            A=np.append(A,np.array(X[:,i]).reshape(-1,1),axis=1)

    Y= np.array(X[:,X.shape[1]-1])
    
    A= np.reshape(A,dim)
    
    return A,Y
    

In [6]:
maxsize = 100, 100

In [7]:
if(os.path.exists(rootpath+'\image-recognition-tensorflow-master\train_images.npy')==False):
    (train_images, train_labels) = load_image_dataset(rootpath+'\image_classification\plastic_paper', maxsize)
    (test_images, test_labels) = load_image_dataset(rootpath+'\image_classification\plastic_paper\test_set', maxsize)

#     (train_images, train_labels) = load_image_dataset(rootpath+'\image_classification\chihuahua-muffin', maxsize)
#     (test_images, test_labels) = load_image_dataset(rootpath+'\image_classification\chihuahua-muffin/test_set', maxsize)
    print(train_images.shape)
    train_images, train_labels = shuffle_examples(train_images, train_labels)
    test_images, test_labels = shuffle_examples(test_images, test_labels)
    print('here')
    data={
        'train_images': np.reshape(train_images,(train_images.shape[0],-1)),
        'train_labels':train_labels.reshape(-1,1),
        'test_images':np.reshape(test_images,(test_images.shape[0],-1)),
        'test_labels':test_labels.reshape(-1,1)
    }
#     save_data(data)
    
else:
    print('found')
    train_images = np.load('train_images.npy')
    train_labels = np.load('train_labels.npy')
    test_images = np.load('test_images.npy')
    test_labels = np.load('test_labels.npy')
    
    train_images= np.reshape(train_images,(train_images.shape[0],maxsize))
    test_images= np.reshape(test_images,(test_images.shape[0],maxsize))



OSError: [WinError 123] The filename, directory name, or volume label syntax is incorrect: 'D:\\hackathon\\image_classification\\plastic_paper\test_set'

In [ ]:
# class_names = ['chihuahua', 'muffin']
class_names = ['plastic', 'paper']




In [ ]:

print(train_images.shape)

In [ ]:

print(train_labels.shape)

In [ ]:
print(test_images.shape)
print(test_labels)

In [ ]:
def display_images(images, labels):
        plt.figure(figsize=(10,10))
        grid_size = min(25, len(images))
        for i in range(grid_size):
                plt.subplot(5, 5, i+1)
                plt.xticks([])
                plt.yticks([])
                plt.grid(False)
                plt.imshow(images[i], cmap=plt.cm.binary)
                plt.xlabel(class_names[labels[i]])

In [ ]:
display_images(train_images, train_labels)
plt.show()

In [ ]:
train_images = train_images / 255.0
test_images = test_images / 255.0
print(train_images.shape)
print(test_images.shape)

In [ ]:
# Setting up the layers.

# model = keras.Sequential([
#     keras.layers.Flatten(input_shape=(100, 100)),
#         keras.layers.Dense(128, activation=tf.nn.sigmoid),
#         keras.layers.Dense(16, activation=tf.nn.sigmoid),
#     keras.layers.Dense(2, activation=tf.nn.softmax)
# ])

model = keras.models.Sequential([
                keras.layers.Flatten(input_shape = ( 100, 100)),
                keras.layers.Dense(512, activation=tf.nn.relu),
                keras.layers.Dropout(0.5),
                keras.layers.Dense(128, activation=tf.nn.relu),
                keras.layers.Dense(16, activation=tf.nn.relu),
        keras.layers.Dense(2, activation=tf.nn.softmax)
        ])

# model = keras.models.Sequential([
#                 keras.layers.Flatten(input_shape = ( 100, 100)),
#                 keras.layers.Dense(1024, activation=tf.nn.relu),
#                 keras.layers.Dropout(0.5),
#                 keras.layers.Dense(512, activation=tf.nn.relu),
#                 keras.layers.Dropout(0.5),
#                 keras.layers.Dense(64, activation=tf.nn.relu),
#                 keras.layers.Dense(16, activation=tf.nn.relu),
#         keras.layers.Dense(2, activation=tf.nn.softmax)
#         ])

In [ ]:
sgd = keras.optimizers.SGD(lr=0.01, decay=1e-5, momentum=0.7, nesterov=True)

model.compile(optimizer=sgd,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(train_images, train_labels, epochs=100)

In [ ]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
print('Test accuracy:', test_acc)

In [ ]:
predictions = model.predict(test_images)

In [ ]:
print(predictions)

In [ ]:
display_images(test_images, np.argmax(predictions, axis = 1))
plt.show()

keep the images you want to classify in 'unknown' folder

In [ ]:
# serialize model to JSON
# model_json = model.to_json()
# with open(rootpath+"\image_classification\model.json", "w") as json_file:
#     json_file.write(model_json)
# # serialize weights to HDF5
# model.save_weights(rootpath+"\image_classification\model.h5")
# print("Saved model to disk")

(test_image, label) = load_image_dataset(rootpath+'/image_classification/unknown', maxsize)

predictons=model.predict(test_image)
print(predictions)
display_images(test_image, np.argmax(predictions, axis = 1))
plt.show()
 